# Gaussian processes

Now that we've encountered a means for mathematically describing the similarity between data we need to develop some method to take this information and perform regression. 

Suppose every observation which we make is drawn from a normal distribution which has some mean value, and some standard deviation.

\\[ y \sim \mathcal{N}(\mu, \sigma) \\]

This means that if we have a whole set of data it will be drawn from a multivariate normal distribution: we'll have a collection of random variables. So for two data

\\[ (y_1, y_2) \sim \mathcal{N}(\vec{\mu}, \Sigma) \\]

where $\Sigma$ is the covariance matrix for the multivariate normal distribution. We can extend this continually to whatever size of data set we have.

We define $\Sigma$ as the covariance of the points $(x_1, x_2)$ where the data $(y_1, y_2)$ are observed, and we end up with a *Gaussian process prior*; for simplicity we normally take the mean to be zero (and this can be ensured by pre-processing our data).